### Regressão de dados em painel 

Pacotes

In [2]:
import pandas as pd
import numpy as np
from itertools import product

In [3]:
panel_data = pd.read_parquet('../Dados/pnad_covid/pnad_covid_tratada.parquet', columns=['semana_dt',
                                                                                        'domicilio_id', 
                                                                                        'UF', 
                                                                                        'A003',
                                                                                        'A004',
                                                                                        'A005', 
                                                                                        'formalidade', 
                                                                                        'setor', 
                                                                                        'Ocupação',
                                                                                        'C013'
                                                                                        ])

In [4]:
panel_data

,semana_dt,domicilio_id,UF,A003,A004,A005,formalidade,setor,Ocupação,C013
0,18,1100000161Mulher221974,Rondônia,Mulher,Parda,Médio completo,Formal,Serviços,NaN,NaN
1,18,1100000161Mulher1521992,Rondônia,Mulher,Parda,Médio completo,Formal,Serviços,NaN,NaN
2,18,1100000161Mulher2412001,Rondônia,Mulher,Parda,Médio completo,NaN,NaN,NaN,NaN
3,18,1100000161Homem29122012,Rondônia,Homem,Parda,Fundamental incompleto,NaN,NaN,NaN,NaN
4,21,11000001610Homem1861976,Rondônia,Homem,Parda,Fundamental incompleto,Informal,Serviços,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
381433,45,5300444346Homem2291962,Distrito Federal,Homem,Parda,Fundamental incompleto,Informal,Agricultura,NaN,NaN
381434,45,5300444346Mulher261965,Distrito Federal,Mulher,Branca,Fundamental incompleto,Informal,Agricultura,NaN,NaN
381435,45,5300444349Homem25121973,Distrito Federal,Homem,Parda,Fundamental incompleto,Informal,Agricultura,NaN,NaN
381436,45,5300444349Mulher2681980,Distrito Federal,Mulher,Parda,Fundamental incompleto,Informal,Agricultura,NaN,NaN


In [3]:
uf_dict = {
    'Acre': 'AC',
    'Alagoas': 'AL',
    'Amapá': 'AP',
    'Amazonas': 'AM',
    'Bahia': 'BA',
    'Ceará': 'CE',
    'Distrito Federal': 'DF',
    'Espírito Santo': 'ES',
    'Goiás': 'GO',
    'Maranhão': 'MA',
    'Mato Grosso': 'MT',
    'Mato Grosso do Sul': 'MS',
    'Minas Gerais': 'MG',
    'Pará': 'PA',
    'Paraíba': 'PB',
    'Paraná': 'PR',
    'Pernambuco': 'PE',
    'Piauí': 'PI',
    'Rio de Janeiro': 'RJ',
    'Rio Grande do Norte': 'RN',
    'Rio Grande do Sul': 'RS',
    'Rondônia': 'RO',
    'Roraima': 'RR',
    'Santa Catarina': 'SC',
    'São Paulo': 'SP',
    'Sergipe': 'SE',
    'Tocantins': 'TO'
}

In [4]:
panel_data['sigla_uf'] = panel_data['UF'].map(uf_dict)

In [5]:
df_datas_flex = pd.read_excel('../Anexos/datas_flexibilização.xlsx')

df_datas_flex['semana'] = pd.to_datetime(df_datas_flex['Data'].values, format='%d/%m/%Y').strftime('%U')

df_datas_flex.head()




#df_datas_flex.set_index('UF', inplace=True)
panel_data = pd.merge(panel_data, df_datas_flex, left_on='sigla_uf', right_on='UF', suffixes=('', '_flex'))
#dummy_flex = pd.get_dummies(pnad_covid_flex['semana'])

In [6]:
panel_data.shape

(2508686, 14)

variáveis de identificação:
- semana_dt = semana de referencia
- domicilio_id = identificação do domicilio

variáveis exógenas:
- A003, sexo
- A004, cor
- A005, escolaridade
- formalidade
- semana (semana de flexibilização em formato dummy)

variável endógena:
C013, trabalho remoto

In [7]:
panel_data = panel_data[['semana_dt', 'domicilio_id', #identificadores
                         'C013', #endógena
                         'A003', 'A004', 'A005', 'formalidade', 'semana']]#exógenas

In [8]:
panel_data.shape

(2508686, 8)

In [9]:
panel_data['C013'].fillna('Não', inplace=True)

In [10]:
#panel_data.dropna(subset='C013', inplace=True)

In [11]:
panel_data.shape

(2508686, 8)

### Balanceamento do modelo

In [11]:
# correcting unbalanced data (like ids missing dates along the dataset)
 
ids = list(panel_data['domicilio_id'].unique())
dates = list(panel_data['semana_dt'].sort_values().unique())

In [12]:
indexes = pd.MultiIndex.from_product(iterables = [ids, dates],names = ['id', 'time'])

In [13]:
balanced_panel = product(ids, dates)

In [14]:
df_aux = pd.DataFrame(data = balanced_panel, columns = ['domicilio_id', 'semana_dt'],index=indexes)

In [15]:
#merge the balanced panel with the original dataset

panel_data = pd.merge(df_aux, panel_data, how = 'left', left_on = ['domicilio_id', 'semana_dt'], right_on = ['domicilio_id', 'semana_dt'])

In [16]:
#set multiindex to the dataset with portfolio_id and date

panel_data.set_index(['domicilio_id', 'semana_dt'], inplace=True)

---

#### Output

In [18]:
#exportar para parquet

panel_data.to_parquet('../Dados/pnad_covid/pnad_covid_balanced.parquet')

---